# Topic Modeling

This notebook wants to explore the topic modeling possibilities of the summaries dataset. We will implement the model using Bertopic.

In [1]:
# Modules to import
import sys
import pandas as pd
import numpy as np

# Paths to add
paths = ['../data','../scripts','../utils']
for path in paths:
    sys.path.append(path)
    
# Data loader
from dataLoader import loadDataframe

# Load data
path_to_directory = '../../data/cleanData/'
df_movies = loadDataframe('movies', path_to_directory)
df_summaries = loadDataframe('summaries', path_to_directory)

c:\Users\arnau\OneDrive\Documents\Scolarité EPFL\ADA\ada-2024-project-teamcsx24\src\models\../data\dataLoader.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_to_convert] = df[columns_to_convert].applymap(eval)


### Bertopic

First step is to preprocess text to remove stopwords, punctuation, and lemmatize the words. We use spacy and nltk for this task. Indeed, these libraries are powerful tools for text preprocessing.

In [2]:
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from tqdm import tqdm

In [3]:
# Load the spaCy model for English language
nlp = spacy.load("en_core_web_sm")

# Download stopwords and punkt tokenizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Load the English stopwords
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arnau\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arnau\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
def preprocess_text(text):
    # Tokenisation
    doc = nlp(text.lower())  # Convert to lowercase
    
    # Lemmatisation and remove stopwords and punctuation
    processed_tokens = [
        token.lemma_ for token in doc if token.text not in stop_words and token.text not in string.punctuation
    ]
    
    char_to_remove = ["'s", " "]
    processed_tokens = [token for token in processed_tokens if token not in char_to_remove]
    
    word_to_remove = ['film','movie','story','tell','leave']
    processed_tokens = [token for token in processed_tokens if token not in word_to_remove]
    
    return " ".join(processed_tokens)

In [5]:
# Apply the preprocessing to the summaries
tqdm.pandas()
df_cleaned_summaries = df_summaries.copy()
df_cleaned_summaries["cleaned_summary"] = df_cleaned_summaries["summary"].progress_apply(preprocess_text)

# Drop the original summary column
df_cleaned_summaries.drop(columns=["summary"], inplace=True)

  0%|          | 0/42303 [00:00<?, ?it/s]

100%|██████████| 42303/42303 [1:20:13<00:00,  8.79it/s]  


All the summaries are not in English, so we use the library `langdetect` to filter out the non-English summaries.

In [6]:
# Detect the language of each summary
from langdetect import detect

def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

df_cleaned_summaries["language"] = df_cleaned_summaries["cleaned_summary"].progress_apply(detect_language)

100%|██████████| 42303/42303 [19:08<00:00, 36.82it/s]


In [7]:
# Save the cleaned summaries
df_cleaned_summaries.to_csv("../../data/topicModelData/cleaned_summaries.csv", index=False)

In [8]:
# Load the cleaned summaries
df_cleaned_summaries = pd.read_csv("../../data/topicModelData/cleaned_summaries.csv")

In [9]:
pourcentage_english = df_cleaned_summaries["language"].value_counts(normalize=True)["en"] * 100
print('Pourcentage of summaries in English: {:.2f}%'.format(pourcentage_english))

Pourcentage of summaries in English: 99.46%


We can just decide to drop the non-English summaries or translate them to English. In this notebook, we will drop the non-English summaries.

In [10]:
df_cleaned_summaries = df_cleaned_summaries[df_cleaned_summaries["language"] == "en"]

In [11]:
docs = df_cleaned_summaries["cleaned_summary"].tolist()

In [12]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from transformers import pipeline
from bertopic.representation import TextGeneration

c:\Users\arnau\anaconda3\envs\ada\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1. Embedding

In [13]:
# Embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode the summaries
embeddings = embedding_model.encode(docs, show_progress_bar=True)

Batches: 100%|██████████| 1315/1315 [42:35<00:00,  1.94s/it] 


In [14]:
# Save the embeddings
np.save("embeddings.npy", embeddings)

In [ ]:
# Load the embeddings
embeddings_model = np.load("embeddings.npy")

2. UMAP

In [15]:
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

3. HDBSCAN

In [16]:
hdbscan_model = HDBSCAN(min_cluster_size=50, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

4. Vectorizer

In [17]:
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))

In [18]:
# Create the model
topic_model = BERTopic(embedding_model=embedding_model,
                       umap_model=umap_model,
                       hdbscan_model=hdbscan_model,
                       vectorizer_model=vectorizer_model,
                       verbose=True)

# Fit the model
topics, probabilities = topic_model.fit_transform(docs)
df_cleaned_summaries['topic'] = topics

2024-11-26 20:49:24,887 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 1315/1315 [16:42:18<00:00, 45.73s/it]       
2024-11-27 13:31:50,447 - BERTopic - Embedding - Completed ✓
2024-11-27 13:31:50,450 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-27 13:33:40,868 - BERTopic - Dimensionality - Completed ✓
2024-11-27 13:33:40,881 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-27 13:34:07,625 - BERTopic - Cluster - Completed ✓
2024-11-27 13:34:07,634 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-11-27 13:34:37,968 - BERTopic - Representation - Completed ✓


In [19]:
# Save the model
topic_model.save("topic_model_min_cluster_size_50")

2024-11-27 13:34:43,258 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [20]:
# Print the topics
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,20926,-1_man_kill_make_try,"[man, kill, make, try, time, life, friend, new...",[backwoodsman name adam pontipee new bride mil...
1,0,5220,0_love_father_marry_family,"[love, father, marry, family, come, son, life,...",[ekam – son soil family relation family disput...
2,1,4606,1_kill_house_man_make,"[kill, house, man, make, car, try, police, hom...",[plot bronx borough new york city entire anglo...
3,2,2213,2_life_love_mother_father,"[life, love, mother, father, family, year, new...",[michelle jordan young energetic 8 year old gi...
4,3,1384,3_murder_police_detective_crime,"[murder, police, detective, crime, killer, pri...",[17 year old orphan henri young steal 5.00 gro...
5,4,1165,4_ho_li_master_man,"[ho, li, master, man, china, wong, chinese, ki...",[start lee ho get arm chop white faced henchma...
6,5,638,5_earth_scientist_planet_human,"[earth, scientist, planet, human, alien, space...",[plot concern legion wing serpent rogue group ...
7,6,477,6_confederate_town_union_man,"[confederate, town, union, man, sheriff, war, ...",[colonel jonas fanatical unrepentant confedera...
8,7,369,7_life_antonio_family_juan,"[life, antonio, family, juan, love, argentina,...",[2000 elsa 25 year old woman barely make livin...
9,8,352,8_earth_ship_planet_alien,"[earth, ship, planet, alien, space, human, cre...",[sometime future earth recover robot war devas...


Our algorithm provides 46 different topics. We have to explore the topics to understand if they make sense and if some are redundant or useless.

In [ ]:
# Visualise topics by dominant words
topic_model.visualize_barchart()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.0066474820312572794, 0.006677281333643151,
                    0.007226114374007955, 0.007938495723986175,
                    0.008736010465547988],
              'xaxis': 'x',
              'y': [house  , man  , kill  , leave  , tell  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.00948352908824549, 0.009995191744878368,
                    0.010838373977140638, 0.010984012781752225,
                    0.014269632815002396],
              'xaxis': 'x2',
              'y': [come  , family  , marry  , father  , love  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.00977709906251261, 0.012087763013324988,
                    0.01305912102422604, 0.01329303707112041, 0.0392759633011321],
              'xaxis': 'x3',
              'y': [woman  , movie  , life  , story  , film  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.010515751404590592, 0.01077423740308684,
                    0.011432968761850014, 0.012776040357816721,
                    0.014254529820195105],
              'xaxis': 'x4',
              'y': [life  , family  , year  , father  , mother  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.010906181469115386, 0.011482938146559037,
                    0.011720931295672404, 0.015063241668068023,
                    0.016714027182201052],
              'xaxis': 'x5',
              'y': [crime  , bank  , detective  , police  , murder  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.007498827769339425, 0.007670552605928332,
                    0.0077363017506846985, 0.00837439502287239,
                    0.00849077600786381],
              'xaxis': 'x6',
              'y': [samurai  , leave  , tell  , tokyo  , kill  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.010311947865818138, 0.01105043273167136,
                    0.011097488917088606, 0.01121495200369034,
                    0.011221077096388112],
              'xaxis': 'x7',
              'y': [kung  , li  , ho  , wong  , master  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.014829397505973767, 0.01577642139522333,
                    0.01780926714346602, 0.01815124634587892, 0.02579843237771683],
              'xaxis': 'x8',
              'y': [alien  , human  , planet  , scientist  , earth  ],
              'yaxis': 'y8'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 0',
                                'x': 0.0875,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 1',
                                'x': 0.36250000000000004,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,

In [ ]:
# Visualise topics on a map
topic_model.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'tell | leave | kill | man | house', 4978],
                                   [1, 'love | father | marry | family | come', 3073],
                                   [2, 'film | story | life | movie | woman', 1025],
                                   [3, 'mother | father | year | family | life', 1009],
                                   [4, 'murder | police | detective | bank | crime', 964],
                                   [5, 'kill | tokyo | tell | leave | samurai', 574],
                                   [6, 'master | wong | ho | li | kung', 528],
                                   [7, 'earth | scientist | planet | human | alien', 482],
                                   [8, 'film | love | story | movie | village', 400],
                                   [9, 'love | antonio | life | pierre | young', 348],
                                   [10, 'earth | ship | planet | space | alien', 336],
                                   [11, 'town | sheriff | outlaw | man | indians', 317],
                                   [12, 'juan | luis | pedro | miguel | leave', 313],
                                   [13, 'king | prince | princess | queen | arthur', 298],
                                   [14, 'soldier | man | order | pilot | japanese', 245],
                                   [15, 'ivan | russian | soviet | moscow | russia', 236],
                                   [16, 'german | nazi | hitler | war | von', 221],
                                   [17, 'team | game | player | coach | play', 205],
                                   [18, 'bug | daffy | porky | elmer | duck', 198],
                                   [19, 'ship | pirate | island | boat | shark', 194],
                                   [20, 'hyun | ji | soo | woo | jin', 193],
                                   [21, 'stooge | moe | curly | shemp | larry', 180],
                                   [22, 'tom | jerry | spike | mouse | cat', 176],
                                   [23, 'nazi | german | war | hitler | germany', 171],
                                   [24, 'santa | christmas | claus | scrooge | franklin', 158],
                                   [25, 'film | african | africa | documentary | israeli', 149],
                                   [26, 'jean | pierre | paris | julien | leave', 144],
                                   [27, 'band | rock | music | album | record', 139],
                                   [28, 'ship | island | captain | pirate | treasure', 109],
                                   [29, 'film | korean | story | japanese | china', 99],
                                   [30, 'charlie | charlie brown | brown | snoopy | charley', 99],
                                   [31, 'donald | mickey | pluto | minnie | pete', 97],
                                   [32, 'vampire | ghost | house | blood | group', 94],
                                   [33, 'mithun | chakraborty | mithun chakraborty | lead role | role',
                                    92],
                                   [34, 'panther | pink | pink panther | horse | man', 83],
                                   [35, 'artagnan | musketeer | louis | athos | king', 82],
                                   [36, 'race | car | herbie | racing | win', 75],
                                   [37, 'college | sorority | school | high school | student', 71],
                                   [38, 'caesar | rome | roman | marcellus | cleopatra', 70],
                                   [39, 'stan | ollie | stan ollie | hardy | laurel', 68],
                                   [40, 'betty | bimbo | boop | betty boop | koko', 61],
                                   [41, 'tarzan | jane | jungle | ape | africa', 59],
                                   [42, 'band | song | record | singer | music', 58],
                                   [43, 'peter | spider man | spider | pete | wendy', 57],
                                   [44, 'harry 

### Review topics

Merge similar topics

Drop non-informative topics

In [59]:
df_cleaned_summaries = pd.read_csv("../../data/topicModelData/summaries_with_topics.csv")
docs = df_cleaned_summaries["cleaned_summary"].tolist()

In [60]:
# Load new model
topic_model = BERTopic.load("topic_model_min_cluster_size_50")
topic_info = topic_model.get_topic_info()

In [61]:
topics = topic_model.get_topics()

Using the HDBSCAN model with a minimum cluster size of 50, we reduce the number of topicsfrom 225 to 46, which represents a more manageable number of topics. Now, can merge similar topics and drop non-informative topics.

In [65]:
# topics to merge

# topics_to_merge = [1,3,8] # love and family
# topics_to_merge = [14,21] # nazi germany
topics_to_merge = 

topic_model.merge_topics(docs, topics_to_merge)

In [66]:
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,23416,-1_film_man_leave_kill,"[film, man, leave, kill, make, tell, life, try...",[norbit albert rice open film narrate orphan t...
1,0,4978,0_tell_leave_kill_man,"[tell, leave, kill, man, house, make, try, car...",[film begin voiceover david comment whenever g...
2,1,4482,1_love_father_marry_family,"[love, father, marry, family, son, come, life,...",[film open man chase heavily armed man take co...
3,2,1025,2_film_story_life_movie,"[film, story, life, movie, woman, young, man, ...",[l'amour fou follow dissolution marriage clair...
4,3,964,3_murder_police_detective_bank,"[murder, police, detective, bank, crime, priso...",[los angeles police department officer kill yo...
5,4,574,4_kill_tokyo_tell_leave,"[kill, tokyo, tell, leave, man, samurai, conan...",[shogo yahagi live appear 1980s metropolitan j...
6,5,528,5_master_wong_ho_li,"[master, wong, ho, li, kung, fu, shaolin, figh...",[leader kong tung clan yen chan ying ruthless ...
7,6,482,6_earth_scientist_planet_human,"[earth, scientist, planet, human, alien, space...",[plot movie begin small tremor begin hill outs...
8,7,392,7_german_nazi_hitler_war,"[german, nazi, hitler, war, germany, von, germ...",[1943 two byelorussian boy dig sand field look...
9,8,348,8_love_life_antonio_pierre,"[love, life, antonio, pierre, young, live, wom...",[film begin 1980 spain political transition di...
